<a href="https://colab.research.google.com/github/anubhavgupta1/Deep-Learning-Using-TensorFlow/blob/master/2_Training_Deep_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Preprocessing**

In [1]:
!ls
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive

import os
os.chdir("drive/")
!ls

sample_data
E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.23-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.23-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0

In [2]:
!pip install tensorflow==1.10.0

     |████████████████████████████████| 58.4MB 72kB/s 
     |████████████████████████████████| 12.2MB 43.5MB/s 
     |████████████████████████████████| 3.3MB 42.0MB/s 
     |████████████████████████████████| 573kB 41.9MB/s 
ERROR: xarray 0.15.1 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.14.5 which is incompatible.
ERROR: tifffile 2020.9.3 has requirement numpy>=1.15.1, but you'll have numpy 1.14.5 which is incompatible.
ERROR: spacy 2.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: seaborn 0.11.0 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: plotnine 0.6.0 has requirement numpy>=1.16.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: pandas 1.1.4 has requirement num

In [3]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# To plot pretty figures
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from functools import partial

from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from matplotlib.colors import ListedColormap


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [4]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [5]:
mnist = input_data.read_data_sets("data/")
X_train = mnist.train.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_test = mnist.test.labels.astype("int")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

In [7]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

What!? That's not a great accuracy for MNIST. Of course, if you train for longer it will get much better accuracy, but with such a shallow network, Batch Norm and ELU are unlikely to have very positive impact: they shine mostly for much deeper nets.

One more thing: notice that the list of trainable variables is shorter than the list of all global variables. This is because the moving averages are non-trainable variables. If you want to reuse a pretrained neural network (see below), you must not forget these non-trainable variables.

In [8]:
# List trainable variables

[v.name for v in tf.trainable_variables()]

[]

In [9]:
# List global variables

[v.name for v in tf.global_variables()]

[]

#**Partial Function**

In [10]:
def multiply(x, y):
    return x*y

In [11]:
def multiplyByTwo_1(x):
    return multiply(x, 2)

In [12]:
multiplyByTwo = partial(multiply, 2)

In [13]:
multiplyByTwo_1(4)

8

In [14]:
multiplyByTwo(4)


8

#**Dummy Example of Standardisation**

In [15]:
x = np.array([0.1,0.01,1,100,200,400,50000,100000])
mean  = np.mean(x)
mean

18837.63875

In [16]:
dx = x - mean
dx

array([-18837.53875, -18837.62875, -18836.63875, -18737.63875,
       -18637.63875, -18437.63875,  31162.36125,  81162.36125])

In [17]:
std = np.sum(dx*dx)/len(x)
std

1207669616.4507608

In [18]:
x_scaled = dx / np.sqrt(std)
x_scaled

array([-0.5420634 , -0.54206599, -0.54203751, -0.53918871, -0.53631114,
       -0.530556  ,  0.89671883,  2.33550394])

#**Dummy Example of Gradient clipping**

In [19]:
a = [(3.4,"a1"),(1.8,"a2"),(-2.9,"a3"),(-3.1,"a4")]
#grads_and_vars = optimizer.compute_gradients(loss)


In [20]:
def clip_by_value(x , th):
  if x > th :
    return th
  if x < -th :
    return -th
  return x

In [21]:
b = []
th = 3
for k , v in a :
  b.append((clip_by_value(k,th), v))
  #capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]
b

[(3, 'a1'), (1.8, 'a2'), (-2.9, 'a3'), (-3, 'a4')]

In [22]:
[(clip_by_value(k,th), v) for k , v in a]
#capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]

[(3, 'a1'), (1.8, 'a2'), (-2.9, 'a3'), (-3, 'a4')]

#**Gradient Clipping**

In [23]:
he_init = tf.contrib.layers.variance_scaling_initializer()

In [24]:
# Create a simple neural net for MNIST and add gradient clipping

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.selu, name="hidden1",kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.selu, name="hidden2",kernel_initializer=he_init)
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.selu, name="hidden3",kernel_initializer=he_init)
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.selu, name="hidden4",kernel_initializer=he_init)
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.selu, name="hidden5",kernel_initializer=he_init)
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

In [25]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [26]:
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

In [27]:
# Now we apply gradient clipping.
# For this, we need to get the gradients, use the `clip_by_value()` function to clip them, then apply them

threshold = 1.0
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var) for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

In [28]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1) #It says whether the targets are in top k predictions
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [29]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [30]:
n_epochs = 40
batch_size = 200

In [31]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images,y: mnist.test.labels})
        print(epoch, "Test accuracy:", accuracy_val)

    save_path = saver.save(sess, "model_ckps7/my_model_final.ckpt")

0 Test accuracy: 0.8988
1 Test accuracy: 0.9211
2 Test accuracy: 0.9282
3 Test accuracy: 0.9366
4 Test accuracy: 0.9409
5 Test accuracy: 0.9428
6 Test accuracy: 0.9464
7 Test accuracy: 0.9486
8 Test accuracy: 0.9494
9 Test accuracy: 0.9523
10 Test accuracy: 0.9539
11 Test accuracy: 0.956
12 Test accuracy: 0.9562
13 Test accuracy: 0.9573
14 Test accuracy: 0.9574
15 Test accuracy: 0.9589
16 Test accuracy: 0.9593
17 Test accuracy: 0.9589
18 Test accuracy: 0.9601
19 Test accuracy: 0.9625
20 Test accuracy: 0.9608
21 Test accuracy: 0.962
22 Test accuracy: 0.9626
23 Test accuracy: 0.9642
24 Test accuracy: 0.9636
25 Test accuracy: 0.9646
26 Test accuracy: 0.9642
27 Test accuracy: 0.9643
28 Test accuracy: 0.965
29 Test accuracy: 0.9659
30 Test accuracy: 0.9674
31 Test accuracy: 0.9664
32 Test accuracy: 0.965
33 Test accuracy: 0.9672
34 Test accuracy: 0.9679
35 Test accuracy: 0.9675
36 Test accuracy: 0.9676
37 Test accuracy: 0.9681
38 Test accuracy: 0.9676
39 Test accuracy: 0.9675


In [32]:
!ls -l model_ckps7/my_model_final.ckpt*

-rw-r--r-- 1 root root 1034840 Dec  6 15:35 model_ckps7/my_model_final.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root     487 Dec  6 15:35 model_ckps7/my_model_final.ckpt.index
-rw-r--r-- 1 root root   54086 Dec  6 15:35 model_ckps7/my_model_final.ckpt.meta
